# Part 5 V2- Explore the Clusters
In this notebook, I will explore the clusters created by Agglomerative​ clustering and create an interactive Folium map of the clusters.  

In [1]:
#!conda install -c conda-forge folium=0.5.0 --yes

In [2]:
#import libraries
import requests
import numpy as np
import pandas as pd
import ibm_boto3
import io

In [3]:
from IPython.display import Image 
from IPython.core.display import HTML 
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib as mpl

In [4]:
# The code was removed by Watson Studio for sharing.

In [5]:
#Import dataframe
obj = resource.Object(bucket_name=bucket, key='VenueDataWithAGGClusters.json').get()
df = pd.read_json(io.BytesIO(obj['Body'].read()))
df.sort_index(inplace=True)
df.head()

,Metropolitan,Country,Continent,Latitude,Longitude,Category 1,Category 2,Category 3,Category 4,Category 5,Category 6,Category 7,Category 8,Category 9,Category 10,Cluster Labels
0,Tokyo,Japan,Asia,35.682839,139.759455,Café,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Lounge,Garden,French Restaurant,Dessert Shop,Thai Restaurant,Steakhouse,1
1,Delhi,India,Asia,28.651718,77.221939,Indian Restaurant,Bar,Café,Flea Market,Lounge,Asian Restaurant,Ice Cream Shop,South Indian Restaurant,Portuguese Restaurant,Plaza,10
2,Mexico City,Mexico,North America,19.432601,-99.133342,Mexican Restaurant,Art Museum,Museum,Arts & Crafts Store,Bar,Ice Cream Shop,Boutique,Bakery,Restaurant,Café,2
3,São Paulo,Brazil,South America,-23.550651,-46.633382,Brazilian Restaurant,Japanese Restaurant,Café,Bakery,Snack Place,Bookstore,Cosmetics Shop,Asian Restaurant,Art Gallery,Chinese Restaurant,1
4,Lagos,Nigeria,Africa,6.455057,3.394179,Lounge,African Restaurant,Bar,Café,Shopping Mall,Pizza Place,Hotel,Art Gallery,Italian Restaurant,Market,1


In [6]:
#Import Venue Count dataframe
obj = resource.Object(bucket_name=bucket, key='VenueCatCount.json').get()
df_cat = pd.read_json(io.BytesIO(obj['Body'].read()))
df_cat.sort_index(inplace=True)
df_cat.head()

,Metro,ATM,Acai House,Accessories Store,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Amphitheater,Antique Shop,...,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Yoshoku Restaurant,Zhejiang Restaurant,Zoo Exhibit
0,Ahmedabad,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Alexandria,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Ankara,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
3,Atlanta,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
4,Bandung,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#How many metro areas are in each cluster?
df_count = df.loc[:,['Metropolitan','Cluster Labels']].groupby('Cluster Labels').count()
df_count

,Metropolitan
Cluster Labels,
0,13
1,29
2,33
3,4
4,4
5,1
6,2
7,1
8,1


In [8]:
#Look at the areas and top categories in each cluster containing more than 2 metro areas in the cluster
dftemp = df.loc[:,['Metropolitan','Cluster Labels']]
df_New = dftemp.join(df_cat.set_index('Metro'), on='Metropolitan')
dfR = df_New.groupby('Cluster Labels', as_index=False).sum()
for i in df_count[df_count['Metropolitan']>2].index:
    print('Cluster ',i)
    print('Top 10 venue categpries:')
    print(*dfR[dfR['Cluster Labels'] == i].drop('Cluster Labels',axis=1).sort_values(by=i, axis = 1, ascending=False).iloc[:,range(10)].columns,sep=', ')
    with pd.option_context('display.max_rows', None):
        display(df[df['Cluster Labels']==i].iloc[:,range(3)])

Cluster  0
Top 10 venue categpries:
Chinese Restaurant, Shopping Mall, Café, Coffee Shop, Japanese Restaurant, Park, Nightclub, Noodle House, Plaza, Lounge


,Metropolitan,Country,Continent
10,Chengdu,China,Asia
12,Shanghai,China,Asia
13,Chongqing,China,Asia
22,Hangzhou,China,Asia
24,Surabaya,Indonesia,Asia
29,Changzhou,China,Asia
33,Nanjing,China,Asia
34,Seoul,South Korea,Asia
45,Guangzhou,China,Asia
46,Qingdao,China,Asia


Cluster  1
Top 10 venue categpries:
Café, Restaurant, Bakery, Japanese Restaurant, Bar, Park, Italian Restaurant, Pizza Place, Coffee Shop, Fast Food Restaurant


,Metropolitan,Country,Continent
0,Tokyo,Japan,Asia
3,São Paulo,Brazil,South America
4,Lagos,Nigeria,Africa
7,Kyoto,Japan,Asia
11,Dhaka,Bangladesh,Asia
14,Karachi,Pakistan,Asia
17,Istanbul,Turkey,Europe
18,Bangkok,Thailand,Asia
19,Kolkata,India,Asia
23,Jakarta,Indonesia,Asia


Cluster  2
Top 10 venue categpries:
Coffee Shop, Café, Italian Restaurant, Bar, Plaza, Cocktail Bar, Mexican Restaurant, Restaurant, Ice Cream Shop, Seafood Restaurant


...

,Metropolitan,Country,Continent
8,Wuhan,China,Asia
37,Jinan,China,Asia
39,Harbin,China,Asia
81,Kanpur,India,Asia


Cluster  4
Top 10 venue categpries:
Café, Bookstore, Coffee Shop, Bar, Sandwich Place, Theater, Dessert Shop, Seafood Restaurant, Restaurant, Persian Restaurant


,Metropolitan,Country,Continent
20,Tehran,Iran,Asia
68,Taipei–Keelung,Taiwan,Asia
85,Ankara,Turkey,Asia
94,Alexandria,Egypt,Africa


Cluster  10
Top 10 venue categpries:
Indian Restaurant, Café, Fast Food Restaurant, Restaurant, Ice Cream Shop, Dessert Shop, Vegetarian / Vegan Restaurant, Bakery, Seafood Restaurant, Bar


,Metropolitan,Country,Continent
1,Delhi,India,Asia
5,Mumbai,India,Asia
28,Bangalore,India,Asia
38,Chennai,India,Asia
57,Hyderabad,India,Asia
61,Pune,India,Asia
75,Ahmedabad,India,Asia
88,Dubai-Sharjah-Ajman,United Arab Emirates,Asia


In [9]:
#Look at the areas in clusters containing 1 or 2 metro areas in the cluster
for i in df_count[df_count['Metropolitan']<=2].index:
    print('Cluster ',i)
    display(df[df['Cluster Labels']==i].iloc[:,range(3)])

Cluster  5


,Metropolitan,Country,Continent
64,Yangon,Myanmar,Asia


Cluster  6


,Metropolitan,Country,Continent
43,Zhengzhou,China,Asia
59,Shenyang,China,Asia


Cluster  7


...

,Metropolitan,Country,Continent
63,Nanchang,China,Asia


Cluster  12


,Metropolitan,Country,Continent
32,Shantou,China,Asia


Cluster  13


,Metropolitan,Country,Continent
16,Tianjin,China,Asia


In [10]:
#Create an interactive folium map with different color markers for each cluster
map_clusters = folium.Map(location=[0,0], zoom_start=2)

#set color scheme for the clusters
map_colors = [colors.rgb2hex(cm.tab20b(i)) for i in range(3,len(df_count.index)+4)]

# add markers to the map
markers_colors = []
for latitude, longitude, metro, country, cluster in zip(df['Latitude'], df['Longitude'], df['Metropolitan'], df['Country'], df['Cluster Labels']):
    label = folium.Popup(str(metro) + ', ' + str(country)+'- Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color=map_colors[cluster-1],
        fill=True,
        fill_color=map_colors[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters.save('map.html')
map_clusters

In [11]:
#Save the map as an html file
#obj = io.open('map.html','rb')
#resource.Bucket(name=bucket).put_object(Key='mapA.html', Body=obj.read())
#obj.close()
#print('Saved')